In [1]:
import pandas as pd

data = pd.read_csv('data/aggregateData_KY.csv')
data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%)
0,Kentucky,Lexington,8/9/2012,55V50,8/12/2013,224,7.01,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481
1,Kentucky,Lexington,8/9/2012,Phoenix,8/12/2013,224,6.97,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481
2,Kentucky,Lexington,8/9/2012,Evermore,8/12/2013,224,6.96,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481
3,Kentucky,Lexington,8/9/2012,4030,8/12/2013,224,6.95,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481
4,Kentucky,Lexington,8/9/2012,Caliber,8/12/2013,224,6.87,368,1,3273.52,804.6,17.607333,12.617071,22.598714,0.313481


In [2]:
from sdv.tabular import CTGAN

model = CTGAN(log_frequency=False, cuda=True, epochs=1000, batch_size=20, generator_lr=.00001, discriminator_lr=.00002)
model.fit(data)

In [3]:
new_data = model.sample(2000)

In [4]:
new_data.to_csv('data/synth_KY_012822_1.csv')
new_data.head()

,State,City,Date Sown,Variety,Date of Cut,Julian Day,Yield (tons/acre),Time Since Sown (Days),Time Since Last Harvest (Days),Total Radiation (MJ/m^2),Total Rainfall (mm),Avg Air Temp (C),Avg Min Temp (C),Avg Max Temp (C),Avg Soil Moisture (%)
0,Kentucky,Lexington,3/31/2015,AFX095005,9/15/2017,261,3.35,286,336,2239.52,770.4,18.731134,15.721390,25.487039,0.283004
1,Kentucky,Lexington,3/31/2015,GA-ALFG-1,9/19/2016,263,5.41,722,382,3640.59,480.4,20.920279,14.143836,27.635996,0.280774
2,Kentucky,Lexington,4/5/2016,Ameristand 403T Plus,8/12/2013,230,6.34,164,18,3239.03,404.0,21.812847,15.604499,29.253107,0.303001
3,Kentucky,Lexington,8/9/2012,Evermore,9/12/2018,254,7.79,838,29,3172.85,986.2,17.607333,13.580732,25.316121,0.290661
4,Kentucky,Lexington,8/9/2012,Evermore,9/16/2015,262,7.79,1304,337,3404.49,518.4,19.294501,16.385746,26.873570,0.284229


In [5]:
# get aggregate data
targetDataLoc = 'data/aggregateData_GAonly_Annual_final.csv'
aggDataLoc = 'data/synth_KY_012822_1.csv'

aggDf = pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [6]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [7]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
               "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
                 "Avg Soil Moisture (%)"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Yield (tons/acre)"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = xDf.reset_index(drop=True)
test_yDf = yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [8]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["mae"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['mae'],
    'model__max_depth': [5,10,25,50,100]
    }

aModelList = [(RandomForestRegressor(), rfParamGrid, "rfTup.pkl"),
              (KNeighborsRegressor(), knnParamGrid, "knnTup.pkl"),
              (SVR(), svrParamGrid, "svrTup.pkl"),
             #(MLPRegressor(), nnParamGrid, "nnTup.pkl")],
             (LinearRegression(), linRegParamGrid, "linRegTup.pkl"),
             (BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             (DecisionTreeRegressor(), dtParamGrid, "dtTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 8 # 11

In [9]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  rfTup
Avg MAE:  3280.553
Avg R squared:  -0.035
Best MAE:  3309.011
Best R squared:  0.013
Parameters of the best model:  {'model__criterion': 'mae', 'model__max_depth': 5, 'model__n_estimators': 100}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
test results on our test data: 
results:
1.5037349
0.12675239411608885
model:  knnTup
Avg MAE:  3424.224
Avg R squared:  -0.099
Best MAE:  3273.619
Best R squared:  -0.042
Parameters of the best model:  {'model__leaf_size': 5, 'model__n_neighbors': 10, 'model__weights': 'distance'}
Features selected by best model:  ['Julian Day', 'Time Since Sown (Days)', 'Total Radiation (MJ/m^2)', 'Total Rainfall (mm)', 'Avg Air Temp (C)', 'Avg Min Temp (C)', 'Avg Max Temp (C)', 'Avg Soil Moisture (%)']
 
test results on our test data: 
results:
0.1636809478826297
0.8984744420588934
mod